In [4]:

from ipymlwidgets import ImageAnnotated
import torch
image = ImageAnnotated(torch.rand((3,32,32)))
display(image)
image.observe(print, "click")
print(image.layers)

xy = torch.randint(image.size[0], (3,2))
boxes = torch.cat([xy, xy + 10, torch.zeros_like(xy[:, :1])], dim=-1)
print(boxes.shape)
image.boxes = boxes
print(boxes)




Image([4, 32, 32])

4
torch.Size([3, 5])
tensor([[ 5, 23, 15, 33,  0],
        [27,  9, 37, 19,  0],
        [ 0, 12, 10, 22,  0]])


In [2]:

from ipymlwidgets.widgets import Image
import torch
image = Image(torch.rand((3,6,10)))
display(image)
image.observe(print, "click")


Image([1, 10, 6])

{'name': 'click', 'old': traitlets.Undefined, 'new': {'x': 7, 'y': 1, 'width': 10, 'height': 6}, 'owner': Image([1, 10, 6]), 'type': 'change'}
{'name': 'click', 'old': {'x': 7, 'y': 1, 'width': 10, 'height': 6}, 'new': {'x': 5, 'y': 2, 'width': 10, 'height': 6}, 'owner': Image([1, 10, 6]), 'type': 'change'}
{'name': 'click', 'old': {'x': 5, 'y': 2, 'width': 10, 'height': 6}, 'new': {'x': 3, 'y': 3, 'width': 10, 'height': 6}, 'owner': Image([1, 10, 6]), 'type': 'change'}
